<a href="https://colab.research.google.com/github/AmandaWu1123/early_warning_system/blob/main/Assignment4_NLP_T5Model/model/Amanda_CareNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Care.csv'

carenotes = pd.read_csv(file_path)
#I use chatGPT to generate the above dataset, 50 carenotes for each students, and there's 10 students in total.
#I make sure that the sentence structure are not repetitive and only few sentences start with the same way

In [ ]:
#Step 1
import re
from sklearn.model_selection import train_test_split


In [ ]:
# Convert Date_Recorded to datetime
carenotes['Date_Recorded'] = pd.to_datetime(carenotes['Date_Recorded'], errors='coerce')

# delete special characters and also strip whitespaces
carenotes['Entry'] = carenotes['Entry'].apply(lambda x: re.sub(r'[^A-Za-z0-9\s,.!?]', '', str(x)).strip())

carenotes['Category'] = carenotes['Category'].str.strip()

# no missinng value
print("Missing Values Before Cleaning:\n", carenotes.isnull().sum())
carenotes.dropna(inplace=True)
print("Shape after dropping missing values:", carenotes.shape)

# reset index
carenotes.reset_index(drop=True, inplace=True)

# Split the data. 80% train, 10% validation, 10% test
train, temp = train_test_split(carenotes, test_size=0.2, random_state=42, shuffle=True)
val, test = train_test_split(temp, test_size=0.5, random_state=42, shuffle=True)

print(f"Train size: {train.shape}")
print(f"Validation size: {val.shape}")
print(f"Test size: {test.shape}")

Missing Values Before Cleaning:
 Student_ID       0
Date_Recorded    0
Category         0
Entry            0
dtype: int64
Shape after dropping missing values: (500, 4)
Train size: (400, 4)
Validation size: (50, 4)
Test size: (50, 4)


In [ ]:
!pip install datasets

In [ ]:
#used t5
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# ✅ Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"

# ✅ Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


# ✅ Auto-summary generator to avoid copying
def auto_summary(text):
    important_phrases = ['struggle', 'improve', 'issue', 'problem', 'succeed', 'progress', 'participate']
    for sentence in text.split('.'):
        for phrase in important_phrases:
            if phrase in sentence.lower():
                return sentence.strip()
    return text.split('.')[0].strip()  # fallback to first sentence if nothing matched

# ✅ Prepare Multi-Task Dataset (with auto-summary)
def create_t5_multitask(df):
    data = []
    for _, row in df.iterrows():
        # Task 1: Boundary Detection
        label = "yes" if ("however" in row['Entry'].lower() or "but" in row['Entry'].lower()) else "no"
        data.append({
            'input_text': f"detect boundary: {row['Entry']}",
            'target_text': label
        })
        # Task 2: Summarization (now condensed)
        data.append({
            'input_text': f"summarize: {row['Entry']}",
            'target_text': auto_summary(row['Entry'])  # ✅ Concise, not full copy
        })
        # Task 3: Title Generation
        data.append({
            'input_text': f"generate title: {row['Entry']}",
            'target_text': f"{row['Category']} Reflection"
        })
    return Dataset.from_pandas(pd.DataFrame(data))

train_dataset = create_t5_multitask(train)
val_dataset = create_t5_multitask(val)

# ✅ Tokenization
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# ✅ Training arguments — optimized
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/model/t5_carenotes_multitask_v2',  #New folder
    num_train_epochs=2,                # Epoch 3 somehow stopps running, so i pick epoch 2
    per_device_train_batch_size=2,    # Avoid OOM
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=20,
    save_total_limit=1,
    fp16=True    #Mixed precision
)

# reate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

# Save to my drive
trainer.save_model('/content/drive/MyDrive/model/t5_carenotes_multitask_v2')
tokenizer.save_pretrained('/content/drive/MyDrive/model/t5_carenotes_multitask_v2')




Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-32-906899464ed0>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.037200,0.012357
2,0.024900,0.006041


('/content/drive/MyDrive/model/t5_carenotes_multitask_v2/tokenizer_config.json',
 '/content/drive/MyDrive/model/t5_carenotes_multitask_v2/special_tokens_map.json',
 '/content/drive/MyDrive/model/t5_carenotes_multitask_v2/spiece.model',
 '/content/drive/MyDrive/model/t5_carenotes_multitask_v2/added_tokens.json')

In [ ]:
#test example:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load model
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/t5_carenotes_multitask')
model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/t5_carenotes_multitask')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Example 1: Summarization Task
sample_entry = "The student struggled at first but improved over time. However, peer issues still impact focus."
summarize_input = f"summarize: {sample_entry}"
inputs = tokenizer(summarize_input, return_tensors="pt").to(model.device)
summary_ids = model.generate(inputs.input_ids, max_length=100)
print("Generated Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))

# Example 2: Boundary Detection Task
boundary_input = f"detect boundary: {sample_entry}"
inputs = tokenizer(boundary_input, return_tensors="pt").to(model.device)
boundary_ids = model.generate(inputs.input_ids, max_length=10)
print("Boundary Prediction:", tokenizer.decode(boundary_ids[0], skip_special_tokens=True))

# Example 3: Title Generation Task
title_input = f"generate title: {sample_entry}"
inputs = tokenizer(title_input, return_tensors="pt").to(model.device)
title_ids = model.generate(inputs.input_ids, max_length=20)
print("Generated Title:", tokenizer.decode(title_ids[0], skip_special_tokens=True))

# BASELINE COMPARISON (Rule-based): Use first sentence as baseline summary
baseline_summary = sample_entry.split('.')[0]
print("Baseline Summary (First Sentence):", baseline_summary)


Generated Summary: The student struggled at first but improved over time. However, peer issues still impact focus.
Boundary Prediction: no
Generated Title: peer issues Reflection
Baseline Summary (First Sentence): The student struggled at first but improved over time


In [ ]:
!pip install rouge_score
!pip install evaluate
import evaluate
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/t5_carenotes_multitask')
model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/t5_carenotes_multitask')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Auto-summary function to generate true reference targets
def auto_summary(text):
    important_phrases = ['struggle', 'improve', 'issue', 'problem', 'succeed', 'progress', 'participate']
    for sentence in text.split('.'):
        for phrase in important_phrases:
            if phrase in sentence.lower():
                return sentence.strip()
    return text.split('.')[0].strip()  # fallback to first sentence if nothing matched

# Initialize ROUGE scorer
rouge = evaluate.load('rouge')

model_summaries = []
baseline_summaries = []
references = []

# Sample 10 random care notes for evaluation
sample_data = carenotes.sample(10, random_state=42)

for _, row in sample_data.iterrows():
    # Reference should be the auto_summary target (NOT full Entry)
    reference = auto_summary(row['Entry'])
    references.append(reference)

    # Model Summary Generation
    test_text = f"summarize: {row['Entry']}"
    inputs = tokenizer(test_text, return_tensors="pt").to(model.device)
    summary_ids = model.generate(inputs.input_ids, max_length=100, num_beams=4)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    model_summaries.append(model_summary)

    # Baseline Summary (First Sentence Rule)
    baseline_summary = row['Entry'].split('.')[0]
    baseline_summaries.append(baseline_summary)

# Compute ROUGE for Model Summaries
model_rouge_scores = rouge.compute(predictions=model_summaries, references=references)
print("Model ROUGE Scores (against correct summary targets):", model_rouge_scores)

# Compute ROUGE for Baseline Summaries
baseline_rouge_scores = rouge.compute(predictions=baseline_summaries, references=references)
print("Baseline ROUGE Scores:", baseline_rouge_scores)


✅ Model ROUGE Scores (against correct summary targets): {'rouge1': np.float64(0.7875800007264538), 'rouge2': np.float64(0.7661776972074454), 'rougeL': np.float64(0.7880693884469627), 'rougeLsum': np.float64(0.7886082658336663)}
✅ Baseline ROUGE Scores: {'rouge1': np.float64(1.0), 'rouge2': np.float64(1.0), 'rougeL': np.float64(1.0), 'rougeLsum': np.float64(1.0)}
